# Vector Autoregression (VAR)

In [282]:
from IPython.display import display, Markdown
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error

# Charging station Dataset

In [283]:
# Load in the dataset
full_data =  pd.read_csv('C:/Users/Sirine/Desktop/Maseer/201801a.txt',sep='\t', encoding="utf-8" ,names = ["Datetime","Time","ChargePointID","ChargePointType","Status","Latitude_Longitude","Address",    "Latitude","Longitude"])


In [284]:
 full_data.tail()

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
1477841,20180110,1535,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1477842,20180110,1555,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1477843,20180110,1635,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1477844,20180110,1705,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1477845,20180128,2030,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [285]:
# Clean empty rows
full_data = full_data.dropna(axis=0)

In [286]:
 full_data.tail()

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
1477732,20180102,1805,CP:CBHM9,CHAdeMO,Occ,"-6.289287,52.678070","Maxol Service Station, Arklow Road, Gorey, Cou...",-6.289287,52.678070
1477733,20180102,1805,CP:C9Q9H,CHAdeMO,OOS,"-6.108436,53.195180","Tesco, Vevay Road, Bray, County Wicklow",-6.108436,53.195180
1477734,20180102,1805,CP:RC07,CHAdeMO,Occ,"-6.232485,54.730728","Texaco, Junction One Shopping Centre, Ballymen...",-6.232485,54.730728
1477735,20180102,1805,CP:RC11,CHAdeMO,OOS,"-7.306255,54.995194","Maxol Service Station, Glendermot Road, Waters...",-7.306255,54.995194
1477736,20180102,1805,CP:RC14,CHAdeMO,Occ,"-5.938419,54.547852","Maxol Service Station, Shaws Bridge, Milltown ...",-5.938419,54.547852


In [287]:
def report(df):
    display(Markdown('<b>head():</b>'))
    display(df.head())
    display(Markdown('<b>describe():</b>'))
    display(df.describe())
    display(Markdown('<b>info():</b>'))
    display(df.info(verbose=True))
    display(Markdown('<b>infer_dtype():</b>'))
    display(df.apply(lambda x: pd.api.types.infer_dtype(x.values)))

In [288]:
report(full_data)

<b>head():</b>

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
0,20180101,1,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332
1,20180101,1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571
2,20180101,1,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911
3,20180101,1,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831
4,20180101,1,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298


<b>describe():</b>

,Datetime,Time,Latitude,Longitude
count,1.449257e+06,1.449257e+06,1.449257e+06,1.449257e+06
mean,2.018012e+07,1.252129e+03,-7.124657e+00,5.346792e+01
std,8.964404e+00,6.324821e+02,1.109551e+00,9.012838e-01
min,2.018010e+07,0.000000e+00,-1.027879e+01,5.155391e+01
25%,2.018011e+07,8.070000e+02,-7.926661e+00,5.286456e+01
50%,2.018012e+07,1.306000e+03,-6.665187e+00,5.335386e+01
75%,2.018012e+07,1.743000e+03,-6.232894e+00,5.432521e+01
max,2.018013e+07,2.359000e+03,-5.547600e+00,5.525138e+01


<b>info():</b>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449257 entries, 0 to 1477736
Data columns (total 9 columns):
Datetime              1449257 non-null int64
Time                  1449257 non-null int64
ChargePointID         1449257 non-null object
ChargePointType       1449257 non-null object
Status                1449257 non-null object
Latitude_Longitude    1449257 non-null object
Address               1449257 non-null object
Latitude              1449257 non-null float64
Longitude             1449257 non-null float64
dtypes: float64(2), int64(2), object(5)
memory usage: 110.6+ MB


None

<b>infer_dtype():</b>

Datetime               integer
Time                   integer
ChargePointID           string
ChargePointType         string
Status                  string
Latitude_Longitude      string
Address                 string
Latitude              floating
Longitude             floating
dtype: object

In [289]:
#convert Datetime to str to combine date (Datetime) and time (Time)
full_data.Datetime = full_data.Datetime.astype(str)

#convert Time to str to combine date (Datetime) and time (Time)
full_data.Time = full_data.Time.astype(str)
#add Time to Datetime column
full_data['Datetime'] = full_data['Datetime'].str.cat(full_data['Time'].values.astype(str), sep='')
#now I don't need the Time column anymone (since it was added to the Datetime column)
 

full_data['Date'] = full_data.Datetime.str.slice(0,8)
full_data['Time'] = full_data.Datetime.str.slice(8)
full_data['Time'] = full_data['Time'].apply(lambda val: '{:0>4}'.format(val))
full_data['Datetime'] = full_data['Date'] + " " + full_data["Time"]

#now let's finally convert Datetime to type datetime in column FormattedDatetime 

full_data['FormattedDatetime'] = pd.to_datetime(full_data.Datetime,format="%Y%m%d %H%M")
del full_data['Datetime']
del full_data['Date']
del full_data['Time']

full_data.head()

,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude,FormattedDatetime
0,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332,2018-01-01 00:01:00
1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571,2018-01-01 00:01:00
2,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911,2018-01-01 00:01:00
3,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831,2018-01-01 00:01:00
4,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298,2018-01-01 00:01:00


In [290]:
report(full_data)

<b>head():</b>

,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude,FormattedDatetime
0,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332,2018-01-01 00:01:00
1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571,2018-01-01 00:01:00
2,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911,2018-01-01 00:01:00
3,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831,2018-01-01 00:01:00
4,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298,2018-01-01 00:01:00


<b>describe():</b>

,Latitude,Longitude
count,1.449257e+06,1.449257e+06
mean,-7.124657e+00,5.346792e+01
std,1.109551e+00,9.012838e-01
min,-1.027879e+01,5.155391e+01
25%,-7.926661e+00,5.286456e+01
50%,-6.665187e+00,5.335386e+01
75%,-6.232894e+00,5.432521e+01
max,-5.547600e+00,5.525138e+01


<b>info():</b>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449257 entries, 0 to 1477736
Data columns (total 8 columns):
ChargePointID         1449257 non-null object
ChargePointType       1449257 non-null object
Status                1449257 non-null object
Latitude_Longitude    1449257 non-null object
Address               1449257 non-null object
Latitude              1449257 non-null float64
Longitude             1449257 non-null float64
FormattedDatetime     1449257 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 99.5+ MB


None

<b>infer_dtype():</b>

ChargePointID           string
ChargePointType         string
Status                  string
Latitude_Longitude      string
Address                 string
Latitude              floating
Longitude             floating
FormattedDatetime     datetime
dtype: object

## Multivariate Time Series (MTS)

A <b>multivariate time series</b> has more than one time-dependent variable just like our dataset. Each variable depends not only on its past values but also has some dependency on other variables. This dependency is used for forecasting future values.

### to_datetime()

In [291]:
full_data['Time'] = pd.to_datetime(full_data.FormattedDatetime, format='%H.%M.%S').dt.time
full_data['Time'].head()

0    00:01:00
1    00:01:00
2    00:01:00
3    00:01:00
4    00:01:00
Name: Time, dtype: object

In [292]:

status_dummies = pd.get_dummies(full_data.Status).astype(int)
status_dummies.head()

,OOC,OOS,Occ,Part
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,1,0,0,0
4,0,0,0,1


In [293]:
#let's reappend the status features to the original dataset
full_data = pd.concat(
    [full_data,status_dummies],
    axis=1) #remember that concatenating columns means axis=1!
#full_data.drop('Status',inplace=True,axis=1) --> this would drop the Status column, but we will want that
full_data.head()

,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude,FormattedDatetime,Time,OOC,OOS,Occ,Part
0,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332,2018-01-01 00:01:00,00:01:00,0,1,0,0
1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571,2018-01-01 00:01:00,00:01:00,1,0,0,0
2,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911,2018-01-01 00:01:00,00:01:00,0,1,0,0
3,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831,2018-01-01 00:01:00,00:01:00,1,0,0,0
4,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298,2018-01-01 00:01:00,00:01:00,0,0,0,1


In [294]:

CS1=full_data.loc[full_data['ChargePointID'] == 'CP:C8PJ7'] 




In [295]:

CS1.FormattedDatetime.min()


Timestamp('2018-01-01 15:20:00')

In [296]:

CS1.FormattedDatetime.max()

Timestamp('2018-01-31 19:52:00')

In [297]:

CS1AVAIL = []
current = []
start = '2018-01-01 15:20:00'
time = pd.to_datetime(start)
#count = 0
while time <= pd.to_datetime('2018-1-31 19:52:00'):
    current.append('CP:C8PJ7') #ChargePointID
    current.append('StandardType2') #ChargePointType
    current.append('A') #Status = available
    current.append('Q-Park Multi-Storey Car Park, Sean McDermott Street, Dublin 1, County Dublin') #Address
    current.append(float(-6.258432)) #Latitude
    current.append(float(53.352022)) #citude
    current.append(pd.to_datetime(time)) #Datetime
    current.append(0) #OOC
    current.append(0) #OOS
    current.append(0) #Occ
    current.append(0) #Part
    CS1AVAIL.append(current)
    current = []
    time = pd.to_datetime(time) + pd.Timedelta(minutes=5)
   
 
 
 


In [298]:


#turn list into df
CS1AVAIL = pd.DataFrame(CS1AVAIL, columns=['ChargePointID',
                                                         'ChargePointType',
                                                         'Status',
                                                         'Address',
                                                         'Latitude',
                                                         'Longitude',
                                                         'FormattedDatetime',
                                                         'OOC',
                                                         'OOS',
                                                         'Occ',
                                                         'Part'])

 


 

In [299]:



"""I was getting an error when trying to update cliftonAVAIL with clifton,
so I reset the index of clifton, which fixed the problem"""
CS1 = CS1.reset_index()  
CS1AVAIL.update(CS1) 

In [300]:
print("CS1AVAIL starts: ", CS1AVAIL.FormattedDatetime.min())
print("CS1AVAIL ends: ", CS1AVAIL.FormattedDatetime.max())

 


CS1AVAIL starts:  2018-01-01 15:20:00
CS1AVAIL ends:  2018-01-31 19:52:00


In [301]:

CS1AVAIL['Status_Number'] = (CS1AVAIL.Status != 'A').astype(int)
  
CS1AVAIL.head()


,ChargePointID,ChargePointType,Status,Address,Latitude,Longitude,FormattedDatetime,OOC,OOS,Occ,Part,Status_Number
0,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
1,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:25:00,0.0,0.0,0.0,1.0,1
2,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:30:00,0.0,0.0,0.0,1.0,1
3,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
4,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:35:00,0.0,0.0,0.0,1.0,1


In [302]:

CS1AVAIL.drop('ChargePointID',inplace=True,axis=1) 
CS1AVAIL.drop('ChargePointType',inplace=True,axis=1) 
CS1AVAIL.drop('Status',inplace=True,axis=1) 
CS1AVAIL.drop('Latitude',inplace=True,axis=1) 
CS1AVAIL.drop('Longitude',inplace=True,axis=1)  
CS1AVAIL.head()

,Address,FormattedDatetime,OOC,OOS,Occ,Part,Status_Number
0,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
1,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:25:00,0.0,0.0,0.0,1.0,1
2,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:30:00,0.0,0.0,0.0,1.0,1
3,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
4,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:35:00,0.0,0.0,0.0,1.0,1


In [303]:
CS1AVAIL.head()

,Address,FormattedDatetime,OOC,OOS,Occ,Part,Status_Number
0,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
1,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:25:00,0.0,0.0,0.0,1.0,1
2,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:30:00,0.0,0.0,0.0,1.0,1
3,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
4,"Q-Park Multi-Storey Car Park, Sean McDermott S...",2018-01-01 15:35:00,0.0,0.0,0.0,1.0,1


In [304]:
CS1AVAIL.drop('Address',inplace=True,axis=1) 


In [321]:

CS1AVAIL.head()

,OOC,OOS,Occ,Part,Status_Number
FormattedDatetime,,,,,
2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
2018-01-01 15:25:00,0.0,0.0,0.0,1.0,1
2018-01-01 15:30:00,0.0,0.0,0.0,1.0,1
2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
2018-01-01 15:35:00,0.0,0.0,0.0,1.0,1


In [324]:
new_data = CS1AVAIL.dropna(axis = 0, how ='any')  

In [325]:
CS1AVAIL.drop('Status_Number',inplace=True,axis=1)  



 

ValueError: labels ['Status_Number'] not contained in axis

In [326]:

new_data.shape

(8695, 4)

In [339]:

CS1AVAIL = CS1AVAIL.set_index('FormattedDatetime')


KeyError: 'FormattedDatetime'

## Stationarity

A stationary time series will more often than not give us a better set of predictions. Similar to the <b>Augmented Dickey-Fuller test</b> for univariate series, we have <b>Johansen’s test</b> for checking the stationarity of any multivariate time series data. Since the test works for only 12 variables, I have randomly dropped in the next iteration, I would drop another and check the eigenvalues.

### coint_johansen()

`statsmodels.tsa.vector_ar.vecm.coint_johansen(endog, det_order, k_ar_diff)` - perform the Johansen cointegration test for determining the cointegration rank of a VECM. 
* <b>endog</b> - the data with presample.
* <b>det_order: int</b> -
  * <b>-1</b> - no deterministic terms.
  * <b>0</b> - constant term.
  * <b>1</b> - linear trend.
* <b>k_ar_diff: int, nonnegative</b> - number of lagged differences in the model.

In [311]:
#from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [312]:
#johan_test_temp = data.drop(['CO(GT)'], axis=1)
#coint_johansen(johan_test_temp, -1, 1).eig

## Train-Validation Split

Creating a validation set for time series problems is tricky because we have to take into account the time component. One cannot directly use the `train_test_split` or k-fold validation since this will disrupt the pattern in the series. The validation set should be created considering the date and time values.

In [327]:
train = CS1AVAIL[:int(0.8*(len(CS1AVAIL)))]
valid = CS1AVAIL[int(0.8*(len(CS1AVAIL))):]

## Vector Autoregression (VAR)

In a <b>VAR</b> model, each variable is a linear function of the past values of itself and the past values of all the other variables. Unlike AR, VAR is able to understand and use the relationship between several variables. This is useful for describing the dynamic behavior of the data and also provides better forecasting results. Additionally, implementing VAR is as simple as using any other univariate technique.

In [328]:
from statsmodels.tsa.vector_ar.var_model import VAR

### VAR()

`statsmodels.tsa.vector_ar.var_model.VAR(endog, exog=None, dates=None, freq=None, missing='none')` - fit VAR(p) process and do lag order selection.
\begin{equation*} 
y_t = A_1y_{t-1} + ... + A_py_{t-p} + u_t
\end{equation*}
* <b>endog</b> - 2-d endogenous response variable. The independent variable.
* <b>exog</b> - 2-d exogenous variable.
* <b>dates</b> - must match number of rows of endog

In [329]:
# Fit the model
model = VAR(endog=train)
model_fit = model.fit()

# Make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [330]:
# Convert array to DataFrame
cols = CS1AVAIL.columns
pred = pd.DataFrame(index=range(0, len(prediction)), columns=[cols])

for j in range(0, len(cols)):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

In [340]:

print(cols)

Index(['OOC', 'OOS', 'Occ', 'Part'], dtype='object')


In [331]:
 
pred = pred.dropna(axis = 0, how ='any')  


In [332]:
# RMSE
for i in cols:
    print('RMSE value for', i, 'is:', sqrt(mean_squared_error(pred[i], valid[i])))

RMSE value for OOC is: 6.902355878913089e-13
RMSE value for OOS is: 0.0
RMSE value for Occ is: 2.0032042855947532e-13
RMSE value for Part is: 2.7677889792880927e-12


In [337]:
# Make final predictions
model = VAR(endog=CS1AVAIL)
model_fit = model.fit()
 

In [338]:

yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[  1.00292092e-15   0.00000000e+00   5.19189356e-16   1.73625442e-15]]
